In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# change the working directory to the Drive root
%cd /content/drive/My\ Drive/Colab\ Notebooks/nlp

/content/drive/My Drive/Colab Notebooks/nlp


# Parsing PDF and text extraction

In [ ]:
# Install libraries
!pip install pandas PyPDF2 python-dotenv requests PyMuPDF langchain langchain-community pdf2image pillow pytesseract
!apt-get install tesseract-ocr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.20
    Uninstalling langchain-0.3.20:
      Successfully uninstalled langchain-0.3.20
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages wil

In [ ]:
import os
import PyPDF2
import fitz  # PyMuPDF
from pdf2image import convert_from_path
from PIL import Image
import pytesseract

# Ensure Tesseract is in your system's PATH or specify the executable path
# pytesseract.pytesseract.tesseract_cmd = r'<full_path_to_your_tesseract_executable>'

def crop_image(element, pageObj):
    """
    Crop an image from a PDF page based on the element's coordinates.
    """
    # Get the coordinates to crop the image from the PDF
    [image_left, image_top, image_right, image_bottom] = [element.x0, element.y0, element.x1, element.y1]

    # Crop the page using coordinates (left, bottom, right, top)
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)

    # Save the cropped page to a new PDF
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    cropped_pdf_path = 'cropped_image.pdf'

    with open(cropped_pdf_path, 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

    return cropped_pdf_path

def convert_to_images(input_file):
    """
    Convert a PDF file to images.
    """
    images = convert_from_path(input_file, dpi=500)
    return images

def image_to_text(image_path):
    """
    Extract text from an image using OCR.
    """
    img = Image.open(image_path)
    text = pytesseract.image_to_string(img)
    return text

def process_pdf(file_path, output_dir):
    """
    Process a PDF file to extract text and images.
    """
    try:
        with open(file_path, 'rb') as pdfFileObj:
            pdfReaded = PyPDF2.PdfReader(pdfFileObj)
            doc = fitz.open(file_path)
            n = doc.page_count
            doc_content = ""

            for i in range(n):
                page_n = doc.load_page(i)
                page_content = page_n.get_text("blocks")

                for element in page_content:
                    if element[6] == 0:  # If the element is text
                        doc_content += element[4]
                    else:  # If the element is an image
                        pageObj = pdfReaded.pages[i]
                        cropped_pdf_path = crop_image(element, pageObj)
                        images = convert_to_images(cropped_pdf_path)
                        if images:
                            images[0].save('temp_image.png', "PNG")
                            image_text = image_to_text('temp_image.png')
                            doc_content += image_text

                doc_content += "\n"

            # Save the extracted content to a text file
            txt_file = os.path.join(output_dir, os.path.basename(file_path).replace('.pdf', '.txt'))
            with open(txt_file, 'w', encoding='utf-8') as file:
                file.write(doc_content)
            print(f"Saved file: {txt_file}")

    except Exception as e:
        print(f"Error processing {file_path}: {e}")

def main():
    data_path = "./data/raw/pdf_data"  # Path to the folder where PDF files are available
    output_dir = "./data/scraped/scraped_pdf_text_data"

    # Ensure output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # Process each PDF file in the directory
    for file in os.listdir(data_path):
        if file.endswith(".pdf"):
            file_path = os.path.join(data_path, file)
            print(f"Processing file: {file_path}")
            process_pdf(file_path, output_dir)

if __name__ == "__main__":
    main()


Processing file: ./data/raw/pdf_data/18398_Annual_Comprehensive_Financial_Report_December_31,_2021.pdf
Saved file: ./data/scraped/scraped_pdf_text_data/18398_Annual_Comprehensive_Financial_Report_December_31,_2021.txt
Processing file: ./data/raw/pdf_data/14147_Pittsburgh_CAR_PDF_Copy.pdf
Saved file: ./data/scraped/scraped_pdf_text_data/14147_Pittsburgh_CAR_PDF_Copy.txt
Processing file: ./data/raw/pdf_data/24770_Pittsburgh,_City_of_ACFR_FINAL_2023.pdf
Saved file: ./data/scraped/scraped_pdf_text_data/24770_Pittsburgh,_City_of_ACFR_FINAL_2023.txt
Processing file: ./data/raw/pdf_data/9623_ISP_Tax_Regulations.pdf
Saved file: ./data/scraped/scraped_pdf_text_data/9623_ISP_Tax_Regulations.txt
Processing file: ./data/raw/pdf_data/23255_2024_Operating_Budget.pdf
Saved file: ./data/scraped/scraped_pdf_text_data/23255_2024_Operating_Budget.txt
Processing file: ./data/raw/pdf_data/9624_Local_Services_Tax_Regulations.pdf
Saved file: ./data/scraped/scraped_pdf_text_data/9624_Local_Services_Tax_Regula

# Scraping Webpage and Check Duplicate Sublink

Parsing static webpages from the collected sublink file. It is a part of the web crawling process.
Based on the URLs of the collected sublink URLs, this script crawls the webpages and extracts ALL the text data (if opening the sublink URL does not timeout).

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import warnings
from urllib.parse import urljoin, urlparse, urlunparse
from urllib3.exceptions import InsecureRequestWarning
from tqdm import tqdm

# Suppress warnings from unverified HTTPS requests
warnings.filterwarnings('ignore', category=InsecureRequestWarning)

# Suppress XMLParsedAsHTMLWarning if parsing XML with HTML parser
from bs4 import XMLParsedAsHTMLWarning
warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

# User agents for simulating browser requests
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
]

# Initialize a session for persistent connection
session = requests.Session()

# Function to fetch the page text with a retry mechanism
def fetch_page_text(url, retries=3, timeout=5):
    """Fetch page text and return the parsed BeautifulSoup object."""
    for attempt in range(retries):
        try:
            headers = {'User-Agent': random.choice(user_agents)}
            response = session.get(url, timeout=timeout, headers=headers, verify=False)
            response.raise_for_status()  # Raise an error for bad responses

            soup = BeautifulSoup(response.content, 'lxml')
            return soup.get_text(separator='\n', strip=True), soup

        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed: {e}")
            time.sleep(1)  # Wait before retrying

    return None, None  # Return None if all attempts fail

# Function to extract all sublinks from the soup object
def extract_sublinks(soup, base_url):
    """Extract and return all sublinks from the HTML soup."""
    sublinks = []
    for link in soup.find_all('a', href=True):
        full_url = urljoin(base_url, link['href'])
        sublinks.append(full_url)
    return sublinks

# Function to save crawled text data to .txt files
def save_crawled_text(url, text, index, folder='./data/scraped/scraped_web_text_data/'):
    """Save crawled text data to individual .txt files."""
    cleaned_text = text.replace('\n', ' ')  # Clean up the text by removing newlines
    output_file = f"{folder}{index}.txt"

    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(cleaned_text)

# Function to save sublinks to a CSV file
def save_sublinks_to_csv(sublinks_data, output_csv):
    """Save sublink data to a CSV file."""
    df = pd.DataFrame(sublinks_data, columns=['Parent URL', 'Sublink'])
    df.to_csv(output_csv, index=False)

# Function to normalize a URL by removing the fragment
def normalize_url(url):
    """Remove the fragment from a URL."""
    parsed_url = urlparse(url)
    return urlunparse(parsed_url._replace(fragment=''))

# Function to check and remove duplicate URLs based on the base URL
def remove_duplicate_urls(file_path, column_name):
    """Remove duplicate URLs from the specified column in the CSV file."""
    data = pd.read_csv(file_path)
    data['normalized_url'] = data[column_name].apply(normalize_url)
    data = data.dropna(subset=['normalized_url'])  # Remove rows with NaN URLs
    data_cleaned = data.drop_duplicates(subset='normalized_url', keep='first')  # Remove duplicates
    data_cleaned = data_cleaned.reset_index(drop=False)  # Keep original index
    data_cleaned['new_index'] = range(len(data_cleaned))  # New index column
    data_cleaned = data_cleaned.drop(columns=['normalized_url'])  # Drop temporary column

    output_file = file_path.replace('.csv', '_filtered.csv')
    data_cleaned.to_csv(output_file, index=False)
    print(f"Duplicates removed. Cleaned data saved to {output_file}")

# Function to crawl through the provided URLs and fetch data
def crawl_urls(url_list):
    """Crawl the list of URLs and return crawled data and sublinks."""
    results = {}
    sublinks_data = []  # List to store (parent_url, sublink) pairs

    for index, url in tqdm(enumerate(url_list), total=len(url_list)):
        print(f"Fetching: {url}")
        text, soup = fetch_page_text(url)
        if text and soup:
            results[url] = text  # Store the crawled text
            sublinks = extract_sublinks(soup, url)  # Extract sublinks
            sublinks_data.extend((url, sublink) for sublink in sublinks)  # Collect sublink pairs
            save_crawled_text(url, text, index)  # Save the crawled text immediately
        else:
            print(f"Failed to parse URL at index: {index}, URL: {url}")

    return results, sublinks_data

# Function to crawl sublinks
def crawl_and_save_sublinks(urls, indexes):
    """Crawl sublinks and save results."""
    for index, url in tqdm(enumerate(urls), total=len(urls)):
        if url.startswith('http'):
            text, _ = fetch_page_text(url)
            if text:
                save_crawled_text(url, text, indexes[index], folder='./data/scraped/scraped_sublink_text_data/')
            else:
                print(f"Failed to parse sublink at index {index}, URL: {url}")

if __name__ == "__main__":
    # Read the CSV file with main URLs
    main_file_path = './data/raw/csv_data/data_source.csv'
    data = pd.read_csv(main_file_path)

    # Extract non-empty URLs from the 'Source URL' column
    main_urls = data[data['Select'] == 'Webpage']['Source URL'].dropna().unique()

    # Start crawling the main URLs
    crawled_data, sublinks_data = crawl_urls(main_urls)

    # Save sublinks to CSV
    output_csv = './data/scraped/parentlink_file_name_url_mapping.csv'
    save_sublinks_to_csv(sublinks_data, output_csv)

    # Remove duplicates from the sublink CSV
    remove_duplicate_urls(output_csv, 'Sublink')

    # Read the CSV file with sublinks for further crawling
    sublink_file_path = './data/scraped/sublink_file_name_url_mapping_filtered.csv'
    sublink_data = pd.read_csv(sublink_file_path)

    # Extract non-empty URLs from the 'Sublink' column
    sublinks = sublink_data['Value'].dropna().unique()
    indexes = sublink_data['new_index']

    # Start crawling the sublinks
    crawl_and_save_sublinks(sublinks, indexes)

    print("Crawling complete!")


  0%|          | 0/156 [00:00<?, ?it/s]

Fetching: https://en.wikipedia.org/wiki/Pittsburgh


  1%|          | 1/156 [00:03<08:52,  3.43s/it]

Fetching: https://en.wikipedia.org/wiki/History_of_Pittsburgh


  1%|▏         | 2/156 [00:04<04:37,  1.80s/it]

...

 10%|█         | 1528/14621 [28:13<6:12:39,  1.71s/it]

Attempt 1 failed: 404 Client Error: Not Found for url: https://www.carnegielibrary.org/exhibit/hname2.html
Attempt 2 failed: 404 Client Error: Not Found for url: https://www.carnegielibrary.org/exhibit/hname2.html
Attempt 3 failed: 404 Client Error: Not Found for url: https://www.carnegielibrary.org/exhibit/hname2.html


 10%|█         | 1529/14621 [28:19<10:38:53,  2.93s/it]

Failed to parse sublink at index 1528, URL: http://www.carnegielibrary.org/exhibit/hname2.html


 10%|█         | 1530/14621 [28:19<8:03:54,  2.22s/it] 

# Dynamic crawling for the events data

In [ ]:
!pip install webdriver_manager
# Update package list and install necessary dependencies
!apt-get update
!apt-get install -y wget unzip libvulkan1

# Download and install Google Chrome
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -f -y

# Install xvfb for virtual framebuffer support
!apt-get install -y xvfb

# Install selenium and chromedriver-autoinstaller
!pip install selenium chromedriver-autoinstaller


Hit:1 http://deb.debian.org/debian buster InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://deb.debian.org/debian buster-updates InRelease
Get:4 https://dl.google.com/linux/chrome/deb stable InRelease [1,825 B]
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://deb.debian.org/debian-security buster/updates InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:13 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:14 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:15 https:/

In [ ]:
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm
import time
# Automatically install the correct version of ChromeDriver
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

# Function to fetch the page content using Selenium
def fetch_page_text_selenium(url, driver):
  try:
      driver.get(url)

      # Wait for the page to load dynamically (up to 10 seconds)
      WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.TAG_NAME, "body"))
      )

      # Get page source
      page_source = driver.page_source

      # Parse the page content using BeautifulSoup
      soup = BeautifulSoup(page_source, 'html.parser')

      # Extract and clean the text from the page
      page_text = soup.get_text(separator='\n', strip=True)
      return page_text

  except Exception as e:
      print(f"Error fetching {url}: {e}")
      return None

# Function to read URLs from CSV and crawl each one
def crawl_urls_from_csv(csv_file_path, url_column_name, driver):
  with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
      reader = csv.DictReader(csvfile)
      urls = [row[url_column_name] for row in reader]  # Extract all URLs

      # Use tqdm to show progress
      for index, url in enumerate(tqdm(urls, desc="Crawling URLs")):
          text = fetch_page_text_selenium(url, driver)
          if text:
              # Save the crawled text to a file with the index as the filename
              output_file = f"./data/scraped/events_test/{index}.txt"
              with open(output_file, 'w', encoding='utf-8') as f:
                  f.write(text)
              print(f"Saved content to {output_file}")

if __name__ == "__main__":
    csv_file_path = './data/raw/csv_data/events_after_10_27.csv'
    url_column_name = 'Source URL'

    # Initialize the Chrome WebDriver
    service = Service(executable_path=r'/usr/bin/chromedriver')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.headless = True
    driver = webdriver.Chrome(service=service, options=options)

    try:
        # Start crawling the URLs from the CSV
        crawl_urls_from_csv(csv_file_path, url_column_name, driver)
    except Exception as e:
        print(f"An error occurred during crawling: {e}")
    finally:
        # Ensure the WebDriver is properly closed
        driver.quit()

Crawling URLs:   2%|▎         | 1/40 [00:02<01:30,  2.31s/it]

Saved content to ./data/scraped/events_test/0.txt


Crawling URLs:   5%|▌         | 2/40 [00:04<01:33,  2.47s/it]

Saved content to ./data/scraped/events_test/1.txt


Crawling URLs:   8%|▊         | 3/40 [00:06<01:24,  2.28s/it]

...

Saved content to ./data/scraped/events_test/37.txt


Crawling URLs:  98%|█████████▊| 39/40 [01:26<00:03,  3.08s/it]

Saved content to ./data/scraped/events_test/38.txt


Crawling URLs: 100%|██████████| 40/40 [01:28<00:00,  2.22s/it]

Saved content to ./data/scraped/events_test/39.txt


# data sharding

In [ ]:
import os
import re
from transformers import BartTokenizer
from tqdm import tqdm

# Load the tokenizer for the BART model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Clean the text
def clean_text(text):
    # Remove non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

# Read the content of a file
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Split text into chunks of 1024 tokens (or fewer)
def split_into_token_chunks(text, max_tokens=1024):
    try:
        # Tokenize the text
        tokens = tokenizer.encode(text, truncation=False, return_tensors="pt")[0]

        # Split tokens into chunks of max_tokens
        chunks = []
        for i in range(0, len(tokens), 1000):
            chunk_tokens = tokens[i:i + 1000]

            # Decode tokens back to text
            chunk_text = tokenizer.decode(chunk_tokens, skip_special_tokens=True)

            # Verify token count for the chunk
            chunk_token_count = len(chunk_tokens)
            if chunk_token_count > max_tokens:
                raise ValueError(f"Chunk exceeds {max_tokens} tokens: {chunk_token_count}")

            if chunk_text.strip():  # Skip empty chunks
                chunks.append(chunk_text)

        return chunks
    except Exception as e:
        print(f"Error tokenizing or splitting text: {e}")
        return []

# Save the processed text to a file
def save_processed_text(text, output_dir, file_name, chunk_index=None):
    if chunk_index is not None:
        file_name = f"{os.path.splitext(file_name)[0]}_chunk_{chunk_index}.txt"
    output_file = os.path.join(output_dir, file_name)
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(text)

# Process each text file in the directory
def process_directory(input_dir, output_dir):
    # Ensure output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Iterate over all text files in the input directory
    for file_name in tqdm(os.listdir(input_dir)):
        if file_name.endswith(".txt"):  # Only process .txt files
            file_path = os.path.join(input_dir, file_name)
            file_base_name = os.path.splitext(file_name)[0]  # Remove file extension

            # Read the file content
            text = read_file(file_path)

            # Skip empty files
            if not text.strip():
                print(f"Skipping empty file: {file_name}")
                continue

            # Clean the text
            cleaned_text = clean_text(text)

            # Skip if cleaned text is empty
            if not cleaned_text.strip():
                print(f"Skipping file with no valid text: {file_name}")
                continue

            # Split the text into chunks of 1024 tokens (or fewer)
            chunks = split_into_token_chunks(cleaned_text, max_tokens=1024)

            # Skip if no valid chunks are produced
            if not chunks:
                print(f"Skipping file with no valid chunks: {file_name}")
                continue

            # Save each chunk as a separate file
            if len(chunks) == 1:
                # If there's only one chunk, save it as the original file name
                save_processed_text(chunks[0], output_dir, file_name)
            else:
                # If there are multiple chunks, save each chunk with a suffix
                for index, chunk in enumerate(chunks):
                    save_processed_text(chunk, output_dir, file_name, chunk_index=index)

    print("All files processed.")

# Set your input and output directories
input_directory = './data/scraped/scraped_web_text_data'  # The directory containing your text files
output_directory = './data/scraped/cleaned_and_truncated_text_data'  # Directory to save processed files

# Create the directory if it doesn't exist
os.makedirs(output_directory, exist_ok=True)

# Process the directory
process_directory(input_directory, output_directory)

  2%|▏         | 3/159 [00:00<00:28,  5.49it/s]

Skipping empty file: 6.txt
Skipping empty file: 2.txt
Skipping empty file: 4.txt
Skipping empty file: 16.txt
Skipping empty file: 15.txt
Skipping empty file: 85.txt
Skipping empty file: 86.txt
Skipping empty file: 87.txt
Skipping empty file: 88.txt
Skipping empty file: 89.txt
Skipping empty file: 91.txt
Skipping empty file: 90.txt
Skipping empty file: 92.txt
Skipping empty file: 94.txt
Skipping empty file: 93.txt
Skipping empty file: 95.txt
Skipping empty file: 96.txt
Skipping empty file: 97.txt
Skipping empty file: 98.txt
Skipping empty file: 120.txt
Skipping empty file: 151.txt
Skipping empty file: 152.txt


100%|██████████| 159/159 [00:16<00:00,  9.81it/s]

All files processed.


In [ ]:
import os
from transformers import pipeline, BartTokenizer
from tqdm import tqdm

# Load the pre-trained summarization model and tokenizer
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0)  # Use GPU (device=0) or CPU (device=-1)

# Read the content of a file
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Summarize a text chunk
def summarize_text(text, max_length=1024):
    try:
        # Skip empty or invalid text
        if not text.strip():
            raise ValueError("Empty text provided")

        # Generate the summary
        summary = summarizer(
            text,
            max_length=max_length,
            min_length=61,
            do_sample=False,
            truncation=True  # Ensure truncation is applied (though text is already truncated)
        )
        return summary[0]['summary_text']
    except Exception as e:
        print(f"Error summarizing text: {e}")
        return None

# Process each shard file in the directory and create summaries
def process_directory(input_dir, output_dir, summary_output_dir, max_summary_length=1024):
    # Ensure the output directories exist
    if not os.path.exists(summary_output_dir):
        os.makedirs(summary_output_dir)

    # Iterate over all text files (shards) in the input directory
    for file_name in tqdm(os.listdir(input_dir)):
        if file_name.endswith(".txt"):  # Only process .txt files
            file_path = os.path.join(input_dir, file_name)
            text = read_file(file_path)

            # Skip empty files
            if not text.strip():
                print(f"Skipping empty file: {file_name}")
                continue

            # Generate the summary
            summary = summarize_text(text, max_length=max_summary_length)
            if not summary:
                print(f"Failed to summarize {file_name}")
                continue

            # Save the summary
            summary_file_path = os.path.join(summary_output_dir, f"{file_name}_summary.txt")
            with open(summary_file_path, 'w', encoding='utf-8') as summary_file:
                summary_file.write(summary)

# Example usage
input_directory = './data/scraped/cleaned_and_truncated_text_data'
summary_output_directory = './data/scraped/output_summary'

process_directory(
    input_directory,
    summary_output_directory,
    summary_output_directory,
    max_summary_length=1024  # Ensure this does not exceed the model's limit
)

Device set to use cuda:0
  0%|          | 0/473 [00:00<?, ?it/s]Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Your max_length is set to 1024, but your input_length is only 1000. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=500)
  2%|▏         | 10/473 [00:17<11:37,  1.51s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Your max_length is set to 1024, but your input_length is only 1000. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=500)
100%|██████████| 473/473 [11:30<00:00,  1.46s/it]
